In [1]:
import torch

# see if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available")

# see if cuDNN is available
if torch.backends.cudnn.enabled:
    print("cuDNN is available")

# set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is set to:", device)

CUDA is available
cuDNN is available
Device is set to: cuda


In [2]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning import Trainer, loggers
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import torchmetrics
import tensorboard

class CIFAR100DataModule(pl.LightningDataModule):
    def __init__(self, data_dir='./data', batch_size=128):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
        ])

    def setup(self, stage=None):
        # Load data
        cifar100_train = datasets.CIFAR100(root=self.data_dir, train=True, download=True, transform=self.transform)
        cifar100_test = datasets.CIFAR100(root=self.data_dir, train=False, download=True, transform=self.transform)

        # Train/val split
        self.train, self.val = random_split(cifar100_train, [45000, 5000])
        self.test = cifar100_test

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, shuffle=True, num_workers=7, persistent_workers=True)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size, num_workers=7, persistent_workers=True)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size, num_workers=7)


In [3]:
import torch
from torch import nn
import torch.nn.functional as F

class InceptionModule(nn.Module):
    def __init__(self, in_channels, c1, c2, c3, c4):
        super(InceptionModule, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_channels, c1, kernel_size=1),
            nn.ReLU()
        )
        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_channels, c2[0], kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1),
            nn.ReLU()
        )
        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_channels, c3[0], kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2),
            nn.ReLU()
        )
        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, c4, kernel_size=1),
            nn.ReLU()
        )

    def forward(self, x):
        return torch.cat([
            self.b1(x),
            self.b2(x),
            self.b3(x),
            self.b4(x)
        ], dim=1)

class GoogLeNet(pl.LightningModule):
    def __init__(self, num_classes=100, learning_rate=0.01):
        super(GoogLeNet, self).__init__()
        self.save_hyperparameters()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.Conv2d(64, 64, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.inception3a = InceptionModule(192, 64, (96, 128), (16, 32), 32)
        self.inception3b = InceptionModule(256, 128, (128, 192), (32, 96), 64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception4a = InceptionModule(480, 192, (96, 208), (16, 48), 64)
        self.inception4b = InceptionModule(512, 160, (112, 224), (24, 64), 64)
        self.inception4c = InceptionModule(512, 128, (128, 256), (24, 64), 64)
        self.inception4d = InceptionModule(512, 112, (144, 288), (32, 64), 64)
        self.inception4e = InceptionModule(528, 256, (160, 320), (32, 128), 128)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception5a = InceptionModule(832, 256, (160, 320), (32, 128), 128)
        self.inception5b = InceptionModule(832, 384, (192, 384), (48, 128), 128)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(1024, num_classes)

        # accuracy:
        self.train_acc = torchmetrics.Accuracy(num_classes=num_classes, average='macro', task='multiclass')
        self.val_acc = torchmetrics.Accuracy(num_classes=num_classes, average='macro', task='multiclass')

        # testing metrics:
        self.test_acc = torchmetrics.Accuracy(num_classes=num_classes, average='macro', task='multiclass')
        self.test_precision = torchmetrics.Precision(num_classes=num_classes, average='macro', task='multiclass')
        self.test_recall = torchmetrics.Recall(num_classes=num_classes, average='macro', task='multiclass')
        self.test_f1 = torchmetrics.F1Score(num_classes=num_classes, average='macro', task='multiclass')
        
    def forward(self, x):
        x = self.stem(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool(x)
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool2(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.train_acc(logits, y)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        self.log('train_acc', self.train_acc, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.val_acc(logits, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', self.val_acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        # Update metrics
        acc = self.test_acc(logits, y)
        precision = self.test_precision(logits, y)
        recall = self.test_recall(logits, y)
        f1 = self.test_f1(logits, y)

        # Log metrics
        self.log('test_loss', loss, on_epoch=True)
        self.log('test_acc', acc, on_epoch=True)
        self.log('test_precision', precision, on_epoch=True)
        self.log('test_recall', recall, on_epoch=True)
        self.log('test_f1', f1, on_epoch=True)
        return {"loss": loss}

In [4]:
early_stopping = EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.00,
    patience =  10,
    verbose = True,
    mode = 'min'
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_acc',
    filename='best-checkpoint',
    save_top_k=1,
    mode='max', 
    verbose=True
)


data_module = CIFAR100DataModule()
model = GoogLeNet(num_classes=100, learning_rate=0.001)
logger = TensorBoardLogger('C:/Users/Owner/Documents/MMA/MULTI TASK/Final Project/', name='cnn_logs')
trainer = pl.Trainer(max_epochs=500, devices=1 if torch.cuda.is_available() else 0, callbacks=[early_stopping, checkpoint_callback])
trainer.fit(model, datamodule=data_module)
trainer.test(model, datamodule=data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name           | Type                | Params
--------------------------------------------------------
0  | stem           | Sequential          | 124 K 
1  | inception3a    | InceptionModule     | 163 K 
2  | inception3b    | InceptionModule     | 388 K 
3  | maxpool        | MaxPool2d           | 0     
4  | inception4a    | InceptionModule     | 376 K 
5  | inception4b    | InceptionModule     | 449 K 
6  | inception4c    | InceptionModule     | 510 K 
7  | inception4d    | InceptionModule     | 605 K 
8  | inception4e    | InceptionModule     | 868 K 
9  | maxpool2       | MaxPool2d           | 0     
10 | inception5a    | InceptionModule     | 1.0 M 
11 | inception5b    | InceptionModule     | 1.4 M 
12 | avgpool        | AdaptiveAvgPool2d   | 0     
13 | flatten        | Flatten             | 0     
14 | fc             | Linear              | 102 K 
15 | train_acc      | MulticlassAccuracy  | 0     
16 | val_acc        | MulticlassA

Epoch 0:   1%|          | 3/352 [00:00<00:58,  6.00it/s, v_num=59, train_loss_step=4.600, train_acc_step=0.0135]

c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


...

Epoch 21: 100%|██████████| 352/352 [00:09<00:00, 38.40it/s, v_num=59, train_loss_step=1.660, train_acc_step=0.404, val_loss=3.510, val_acc=0.260, train_loss_epoch=1.780, train_acc_epoch=0.498]
Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing DataLoader 0: 100%|██████████| 79/79 [00:01<00:00, 57.33it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.22750820219516754    │
│          test_f1          │    0.2082616239786148     │
│         test_loss         │     3.380080461502075     │
│      test_precision       │    0.22547651827335358    │
│        test_recall        │    0.22750820219516754    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 3.380080461502075,
  'test_acc': 0.22750820219516754,
  'test_precision': 0.22547651827335358,
  'test_recall': 0.22750820219516754,
  'test_f1': 0.2082616239786148}]